In [1]:
import xgboost as xgb
import sys,random
import pickle
import os
import numpy as np

In [2]:
# import modin.pandas as pd
import pandas as pd

In [3]:
if not os.path.exists('featurescore'):
    os.mkdir('featurescore')
if not os.path.exists('model'):
    os.mkdir('model')
if not os.path.exists('preds'):
    os.mkdir('preds')

In [4]:
%%time
data_date = pd.read_csv('../../preprocess_data/train_x_date.csv').drop(columns=['id','loan_hour'])
data_raw = pd.read_csv('../../preprocess_data_new/train_ax_nodup.csv',nrows=33465).drop(columns=['id','loan_dt','tag'])
# data_raw = pd.read_csv('../../preprocess_data/train_x_33465.csv').drop(columns=['id','loan_dt','tag'])
# data_null = pd.read_csv('../../preprocess_data/train_x_null.csv').drop(columns=['id'])
data_null = pd.read_csv('../../preprocess_data_new/train_ax_row_null.csv',nrows=33465).drop(columns=['id'])
data_tag = pd.read_csv('../../preprocess_data/train_x_33465.csv',usecols=['tag'])


CPU times: user 59.5 s, sys: 2.42 s, total: 1min 1s
Wall time: 1min 1s


In [5]:
# data_raw_nan = data_raw.dropna(axis=1,how='all')

In [6]:
%%time
data = pd.concat([data_date,data_raw,data_null,data_tag],axis=1)

CPU times: user 2.56 s, sys: 68 ms, total: 2.63 s
Wall time: 2.63 s


In [7]:
%%time
data_label = pd.read_csv('../../preprocess_data/train_y_33465.csv',usecols=['label'])

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.29 ms


In [8]:
data.shape

(33465, 6702)

使用旧数据集 train集的维度为：  
使用新数据集 train集的维度为：6702

## 交叉验证

### 1.PCA降维

In [46]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99).fit(data.fillna(0).values)
data_pca = pca.transform(data.fillna(0).values)

In [19]:
data_pca.shape

In [14]:
from sklearn.model_selection import train_test_split # 分割数据模块
from sklearn.metrics import roc_curve
from sklearn import metrics

#1.分割数据

# x的挑选
# a.基于fscore
# feature_score = pd.read_csv('./feature_score.csv')
feature_names = data.columns
# feature_names = [x for x in feature_names if x!='tag']
x= data[feature_names].values
# b.基于PCA
# x=data_pca
y=data_label.values
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state=3096)
#2.建立模型
params={
    'booster':'gbtree',
    'objective': 'binary:logistic',
    'early_stopping_rounds':100,
    'scale_pos_weight': float(len(data_label)-np.sum(data_label.values))/float(np.sum(data_label.values)),  # 负例样本除以正例样本
    'eval_metric': 'auc',
    'gamma':1,
    'max_depth':6,
    'lambda':1,
    'subsample':0.9,
    'colsample_bytree':0.9,
    'min_child_weight':1, 
    'eta': 0.04,
    'seed':2010,
    'nthread':16
        }
#3.训练模型
# a.基于fscore
# dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=feature_names)
# b.基于PCA
dtrain = xgb.DMatrix(X_train, label=y_train)
# watchlist  = [(dtrain,'train')]
print('开始训练！')
model  = xgb.train(params,dtrain,num_boost_round=300)

开始训练！


In [15]:
# 4.预测结果
print('开始预测！')
# a.基于fscore
# dtest = xgb.DMatrix(X_test, feature_names=feature_names)
# b.基于PCA
# dtest = xgb.DMatrix(X_test)
# c.基于LDA
dtest = xgb.DMatrix(X_test)
y_pre = model.predict(dtest)
auc = metrics.roc_auc_score(y_test, y_pre)
print('AUC:',auc)

开始预测！
AUC: 0.9397646211320003


raw AUC: 0.8130921791239684  
raw 去除nan列 AUC: 0.8125236195981279  
raw 去除nan列 + 统计null AUC: 0.8263001992035767   
nodup + null AUC: 0.8261527677046496  
nodup + null + tag AUC:0.8296696198580618  
nodup + null + tag (前4000维)AUC: 0.8251877022906782    
nodup + null + tag (前5990维) AUC: 0.821991126741565  
nodup + null + tag (前3000维) AUC: 0.8275583356303323  
nodup + null + tag (前2000维) AUC: 0.8249837418081847

nodup + null + tag (PCA降至8维) AUC: 0.5830606257872517

In [32]:
feature_score  = model.get_fscore()
feature_rank = sorted(feature_score,key=lambda x:x[1],reverse=True)
import pickle
pickle.dump(feature_rank,open('./cv_feat_rank.pkl','wb'))

In [8]:
# import pickle
# feature_rank = pickle.load(open('./cv_feat_rank.pkl','rb'))
# feature_rank

In [11]:
import gc
del dtrain,dtest,model
gc.collect()

14

## 训练与预测

In [12]:
%%time
valid_date = pd.read_csv('../../preprocess_data/valid_date.csv').drop(columns=['id','loan_hour'])
# valid_raw = pd.read_csv('../../preprocess_data/valid.csv').drop(columns=['id','loan_dt'])
valid_raw = pd.read_csv('../../preprocess_data_new/valid_nodup.csv').drop(columns=['id','loan_dt'])
valid_null = pd.read_csv('../../preprocess_data_new/valid_row_null.csv').drop(columns=['id'])
valid_tag = pd.read_csv('../predict_tag/tag.csv',usecols=['tag'])

CPU times: user 28.7 s, sys: 1.19 s, total: 29.9 s
Wall time: 32.1 s


In [13]:
%%time
valid = pd.concat([valid_date,valid_raw,valid_null,valid_tag],axis=1)

CPU times: user 1.29 s, sys: 20 ms, total: 1.31 s
Wall time: 1.31 s


In [15]:
dtrain = xgb.DMatrix(data.values, data_label.values, feature_names=list(data.columns))
dtest = xgb.DMatrix(valid.values, feature_names=list(valid.columns))

In [16]:
test_id = pd.read_csv('../../preprocess_data_new/valid_date.csv',usecols=['id']).values

In [17]:
def pipeline(dtrain,dtest,test_id,iteration,random_seed,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight):
    if max_depth==6:
        num_boost_round = 500
    elif max_depth==7:
        num_boost_round = 400
    elif max_depth==8:
        num_boost_round = 300
    
    params={
    'booster':'gbtree',
    'objective': 'binary:logistic',
    'early_stopping_rounds':100,
    'scale_pos_weight': float(len(data_label)-np.sum(data_label.values))/float(np.sum(data_label.values)),  # 负例样本除以正例样本
    'eval_metric': 'auc',
    'gamma':gamma,
    'max_depth':max_depth,
    'lambda':lambd,
    'subsample':subsample,
    'colsample_bytree':colsample_bytree,
    'min_child_weight':min_child_weight, 
    'eta': 0.04,
    'seed':random_seed,
    'nthread':16
        }
    watchlist  = [(dtrain,'train')]
    model = xgb.train(params,dtrain,num_boost_round=num_boost_round,evals=watchlist)
    model.save_model('./model/xgb{0}.model'.format(iteration))
    
    #predict test set
    test_y = model.predict(dtest)
    test_result = pd.DataFrame(test_id,columns=["id"])
    test_result['score'] = test_y
    test_result.to_csv("./preds/xgb{0}.csv".format(iteration),index=None,encoding='utf-8')
    
    #get feature score
    feature_score = model.get_fscore()
    feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
    fs = []
    for (key,value) in feature_score:
        fs.append("{0},{1}\n".format(key,value))
    
    with open('./featurescore/feature_score_{0}.csv'.format(iteration),'w') as f:
        f.writelines("feature,score\n")
        f.writelines(fs)
    

In [18]:
%%time
random_seed = list(range(1000,2000,20))
gamma = [i/1000.0 for i in range(100,200,2)]
max_depth = [6,7,8]
lambd = list(range(200,400,2))
subsample = [i/1000.0 for i in range(600,700,2)]
colsample_bytree = [i/1000.0 for i in range(250,350,2)]
min_child_weight = [i/1000.0 for i in range(200,300,2)]
random.shuffle(random_seed)
random.shuffle(gamma)
random.shuffle(max_depth)
random.shuffle(lambd)
random.shuffle(subsample)
random.shuffle(colsample_bytree)
random.shuffle(min_child_weight)

with open('params.pkl','wb') as f:
    pickle.dump((random_seed,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight),f)

#to reproduce my result, uncomment following lines
"""
with open('params_for_reproducing.pkl','rb') as f:
    random_seed,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight = pickle.load(f)    
"""

for i in range(36):
    pipeline(dtrain,dtest,test_id,i,random_seed[i],gamma[i],max_depth[i%3],lambd[i],subsample[i],colsample_bytree[i],min_child_weight[i])

[0]	train-auc:0.782696
[1]	train-auc:0.812435
[2]	train-auc:0.823735
[3]	train-auc:0.833812
[4]	train-auc:0.843099
[5]	train-auc:0.848643
[6]	train-auc:0.853571
[7]	train-auc:0.856744
[8]	train-auc:0.85972
[9]	train-auc:0.861156
[10]	train-auc:0.861093
[11]	train-auc:0.863894
[12]	train-auc:0.865728
[13]	train-auc:0.867732
[14]	train-auc:0.868753
[15]	train-auc:0.870028
[16]	train-auc:0.871589
[17]	train-auc:0.873765
[18]	train-auc:0.875382
[19]	train-auc:0.876612
[20]	train-auc:0.877214
[21]	train-auc:0.877483
[22]	train-auc:0.878411
[23]	train-auc:0.879437
[24]	train-auc:0.881283
[25]	train-auc:0.882187
[26]	train-auc:0.883197
[27]	train-auc:0.884305
[28]	train-auc:0.885343
[29]	train-auc:0.886068
[30]	train-auc:0.886835
[31]	train-auc:0.887571
[32]	train-auc:0.888806
[33]	train-auc:0.889727
[34]	train-auc:0.891023
[35]	train-auc:0.892245
[36]	train-auc:0.893494
[37]	train-auc:0.894115
[38]	train-auc:0.895063
[39]	train-auc:0.89581
[40]	train-auc:0.896551
[41]	train-auc:0.897504
[42]

raw 去除nan列 + 统计null AUC: 0.82751113123698  
nodup + null + tag: 0.82803008109167

In [27]:
data.shape

(33465, 6702)